In [ ]:
# Compute the derivative of N(t)
deriv_Nt(nt, K, r) = r * nt * (1-nt/K)

#型を指定するならこうやって書ける
# function deriv_Nt(nt::Float64, K::Float64, r::Float64)::Float64
#     return r * nt * (1-nt/K)
# end

# Compute N(t + h)
next_Nt(nt, K, r, h) = nt + h * deriv_Nt(nt, K, r)

In [ ]:
methods(deriv_Nt)

In [ ]:
# global変数として、各種パラメータを定義
tmax = 10.0 # 終了時刻
r = 1.0 # パラメータ1
K = 10.0 #パラメータ2
N0 = 1.0 # 人口の初期値
numtimes  = 100 # 0 ≦ t < tmaxの分割数

h = tmax/numtimes

# r = 1とするとjuliaがrを整数として認識するする。
#loablな変数には型を指定できないので、r::Float64ともできない。

In [ ]:
deriv_Nt(N0, K, r)

In [ ]:
@code_warntype deriv_Nt(N0, K, r)

In [ ]:
@code_warntype next_Nt(N0, K, r, h)

In [ ]:
results = Vector{Float64}(undef, numtimes+1)
#result = zeros(Float64, numtimes+1)

# 初期値
results[1] = N0

In [ ]:
results

In [ ]:
# 時間発展
for t in 1:numtimes
    results[t+1] = next_Nt(results[t], K, r, h)
end

In [ ]:
results

In [ ]:
using Plots

times = LinRange(0, tmax, numtimes+1)
#LinRangeは初期値と最大値を指定して、その間をメッシュで切る。

plot(times, results, label="Numerical", xlabel="time")

In [ ]:
times

In [ ]:
collect(times)
#collectで配列に直す。
#Plotsは配列以外の値も受け取るので、今回の場合には問題ない。

In [ ]:
exact_Nt(time) = K/(1+(K-N0) / N0 * exp(-r*time))

In [ ]:
exact_Nt.(times)
#exact_Ntは配列を受け取るようには設定されていないが、要素ごとに勝手にまとめてくれる。

In [ ]:
err = abs.(result .- exact_Nt.(times))
#Floatの配列のたいしてabsは対応していないので、.をつける。

p = plot(yaxis=:log, ylims=(1e-10, 1000),  xlabel="time")
#pはplotのobjectである。そこにpplot!を2つ追加している。
plot!(p, times, results, label="Numerical")
plot!(p, times, abs.(results .- exact_Nt.(times)), marker=:x, label="Error")

グラフで凹んでいる部分があるのは、errの符号が反転しているから。

In [ ]:
:x
#シンボルオブジェクト

In [ ]:
typeof(:x)

In [ ]:
a = "x"
b = "x"
#メモリ上にはxという値を持ったものが2つある?

In [ ]:
objectid(a)

In [ ]:
objectid(b)

In [ ]:
using DifferentialEquations

# ロジスティック方程式の定義
function logistic!(du, u, params, t)
    r, K = params
    du[1] = r * u[1] * (1 - u[1]/K)
end

# パラメータの設定
params = (r, K)

# 初期条件の設定
u0 = [1.0]  # 開始時点の人口サイズ

# 時間範囲の設定
tspan = (0.0, tmax)

# 問題の設定
prob = ODEProblem(logistic!, u0, tspan, params)

# 微分方程式の解 (5次のルンゲクッタ法)
sol = solve(prob, Tsit5(), abstol=1e-8, reltol=1e-8)
;

sol = solve(解きたい問題, solver, 絶対誤差, 相対誤差)

In [ ]:
sol.t

In [ ]:
sol.u
#配列の配列、sol.u[1]とすると1つ目の配列を取り出せる。

In [ ]:
sol.u[1]

In [ ]:
# 解のプロット

results_de = [u_[1] for u_ in sol.u]
#sol.uの各要素の1つ目を取り出して配列にする。


p = plot(yaxis=:log, ylims=(1e-10, 1000),  xlabel="time")
plot!(p, sol.t, results_de, marker=:o)
plot!(p, sol.t, abs.(results_de .- exact_Nt.(sol.t)), label="error")

中間課題(来週の講義終了時まで)
logistic.ipynbとproject.tomlをgitにpushする。
#manifest.tomlは基本的に環境によるので、gitにpushしない。
出力を含めたままgitにpushするのはどうなのか？(環境によって出力される変数も変わってくる)
→その場合は「全ての出力をクリア」してからpushすれば良い。